<a href="https://colab.research.google.com/github/eternityduck/ML_KPI/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip install roman
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from google.colab import drive
import requests
from roman import toRoman
from sklearn.feature_extraction.text import TfidfVectorizer



text = requests.get('https://www.gutenberg.org/files/11/11-0.txt').text
text = text[1517:]
print(text[:500])



## Pipeline

In [ ]:
import re
regex = re.compile('[^a-zA-Z \n]')

divided_text = []
for i in range(2, 13):
  app, text = text.split(f'CHAPTER {toRoman(i)}.')
  divided_text.append(regex.sub('', app))
divided_text.append(regex.sub('', text.split('THE END')[0]))

## TF-IDF by scikit


In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
vectors = vectorizer.fit_transform(divided_text)
features = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()

pd.set_option('display.max_columns', 21)
pd.set_option('display.width', 220)

df = pd.DataFrame(denselist, columns=features, dtype="float32")
for i in range(12):
  print(df.sort_values(by=i, axis=1, ascending=False), '\n')

## LDA from scikit

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


topwords = []
for i in range(12):
  count_vect = CountVectorizer(stop_words='english')
  doc_term_matrix = count_vect.fit_transform([divided_text[i]])
  LDA = LatentDirichletAllocation(n_components=1, random_state=1)
  LDA.fit(doc_term_matrix)

  ntopwords = 20
  tf_feature_names = count_vect.get_feature_names_out()

  for topic in LDA.components_:
    topwords.append([tf_feature_names[i] for i in topic.argsort()[:-ntopwords - 1:-1]])

for words in topwords:
  print(words)

#Conclusion
In our experiments, we experimented with searching top words in book's chapters by removing unnecessary parts of text to analyse and applying two algorithms, one is statistical and second one is neural.

TF_IDF is statistical and thus is very straight forward and precise giving just the coefficient of the word usage.

LDA is neural algorithm and thus can be imprecise but all depends on training and has more flexibility to be trained on detecting topic words.